<a href="https://colab.research.google.com/github/rajindermath/Crop-Classification/blob/master/23_02_2021_Detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Detectron2 Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/06-02-2021-NEW-SMB

/content/drive/MyDrive/06-02-2021-NEW-SMB


In [ ]:
!ls

10.png	3.png  6.png  9.png		output	 test.json
1.png	4.png  7.png  coco_eval		output1  train
2.png	5.png  8.png  inference_images	test	 train.json


In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 33.2MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████| 276kB 16.7MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=ef18aaa711dccbc4e8396af3ad0b01c6087c77ee295807085ce2917c64491961
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pi

In [ ]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2MB 705kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210204-cp36-none-any.whl size=44945 sha256=bc4c09f75ee71a7645aa2dfd1b691eff77cec5132c3858215d3c118371814641
  Stored in directory: /root/.cache/pip/wheels/7a/d5/3d/1393f94e0a0c6345f674387da5bd382c9aba98a23371a3804e
  Created wheel for iopath: filename=iopath-0.1.3-cp36-none-any.whl size=11169 sha256=b1f06d28f55a06d1bac73e857f215ed8e7b3327d2292b605f750436b78ca5b63
  Stored in directory: /root/.cache/pip/wheels/a9/1d/55/94a55e032409ac7617f9cbb88a0fa2cf4e7208806c29730804
Successfully built fvcore iopath


In [ ]:
exit(0)

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



# Import and Register Custom Detectron2 Data

In [ ]:
#!curl -L "https://app.roboflow.com/ds/Crc5TdkF4m?key=TKcB1tcPjH" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip


In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/06-02-2021-NEW-SMB/train.json", "/content/drive/MyDrive/06-02-2021-NEW-SMB/train")
#register_coco_instances("my_dataset_val", {}, "/content/valid/_annotations.coco.json", "/content/valid")
register_coco_instances("my_dataset_test", {}, "/content/drive/MyDrive/06-02-2021-NEW-SMB/test.json", "/content/drive/MyDrive/06-02-2021-NEW-SMB/test")


In [ ]:
#visualize training data
import matplotlib.pyplot as plt
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    v = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.15)
    v = v.draw_dataset_dict(d)
    fig= plt.figure(figsize = (14, 10))
    plt.axis('off')
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    fig.savefig("/content/drive/MyDrive/06-02-2021-NEW-SMB/8-2-2021-results/15.png", bbox_inches = 'tight', dpi=300)
    plt.show()
    

# Train Custom Detectron2 Detector

In [ ]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
#from .detectron2.tools.train_net import Trainer
#from detectron2.engine import DefaultTrainer
# select from modelzoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines

from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_test",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001


cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05




cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4 #your number of classes + 1

cfg.TEST.EVAL_PERIOD = 500


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[02/07 19:57:01 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.


[02/07 19:57:01 d2.engine.train_loop]: Starting training from iteration 0
[02/07 19:58:00 d2.utils.events]:  eta: 0:22:55  iter: 19  total_loss: 2.747  loss_cls: 1.685  loss_box_reg: 0.969  loss_rpn_cls: 0.045  loss_rpn_loc: 0.014  time: 2.9302  data_time: 0.1191  lr: 0.000020  max_mem: 11797M
[02/07 19:58:54 d2.utils.events]:  eta: 0:20:26  iter: 39  total_loss: 2.286  loss_cls: 1.245  loss_box_reg: 0.971  loss_rpn_cls: 0.053  loss_rpn_loc: 0.017  time: 2.8077  data_time: 0.0101  lr: 0.000040  max_mem: 12065M
[02/07 19:59:50 d2.utils.events]:  eta: 0:19:49  iter: 59  total_loss: 1.925  loss_cls: 0.815  loss_box_reg: 0.993  loss_rpn_cls: 0.031  loss_rpn_loc: 0.012  time: 2.7960  data_time: 0.0110  lr: 0.000060  max_mem: 12065M
[02/07 20:00:44 d2.utils.events]:  eta: 0:18:48  iter: 79  total_loss: 1.689  loss_cls: 0.712  loss_box_reg: 0.929  loss_rpn_cls: 0.018  loss_rpn_loc: 0.012  time: 2.7752  data_time: 0.0109  lr: 0.000080  max_mem: 12065M
[02/07 20:01:38 d2.utils.events]:  eta: 0:

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

[02/07 20:23:41 d2.data.datasets.coco]: Loaded 70 images in COCO format from /content/drive/MyDrive/06-02-2021-NEW-SMB/test.json
[02/07 20:23:41 d2.data.common]: Serializing 70 elements to byte tensors and concatenating them all ...
[02/07 20:23:41 d2.data.common]: Serialized dataset takes 0.13 MiB
[02/07 20:23:41 d2.evaluation.evaluator]: Start inference on 70 images
[02/07 20:23:46 d2.evaluation.evaluator]: Inference done 11/70. 0.2618 s / img. ETA=0:00:17
[02/07 20:23:52 d2.evaluation.evaluator]: Inference done 29/70. 0.2612 s / img. ETA=0:00:12
[02/07 20:23:57 d2.evaluation.evaluator]: Inference done 46/70. 0.2622 s / img. ETA=0:00:07
[02/07 20:24:02 d2.evaluation.evaluator]: Inference done 64/70. 0.2636 s / img. ETA=0:00:01
[02/07 20:24:04 d2.evaluation.evaluator]: Total inference time: 0:00:19.270150 (0.296464 s / img per device, on 1 devices)
[02/07 20:24:04 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:17 (0.262379 s / img per device, on 1 devices)
[02/07 20

OrderedDict([('bbox',
              {'AP': 49.58904061675666,
               'AP-Big': 58.00849433786771,
               'AP-Medium': 42.123593642966426,
               'AP-Small': 48.63503386943585,
               'AP50': 66.20751428073139,
               'AP75': 56.13894904406853,
               'APl': 49.59063708850046,
               'APm': nan,
               'APs': nan})])

# Inference with Detectron2 Saved Weights



In [ ]:
%ls ./output/

coco_instances_results.json                        last_checkpoint
events.out.tfevents.1612726350.bd99ec26bf2b.487.0  metrics.json
events.out.tfevents.1612727821.bd99ec26bf2b.487.1  model_final.pth
instances_predictions.pth


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
#test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
from detectron2.utils.visualizer import ColorMode
from detectron2.data import DatasetCatalog
dataset_dicts = DatasetCatalog.get("my_dataset_train")
for d in random.sample(dataset_dicts,1):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=my_dataset_train_metadata, 
                   scale=0.15, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.axis('off')
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()
    #fig.savefig("/content/drive/MyDrive/06-02-2021-NEW-SMB/inference_images/10.png", bbox_inches = 'tight', dpi=300)
    


ModuleNotFoundError: ignored

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/drive/MyDrive/06-02-2021-NEW-SMB/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])